# Load Module 

In [15]:
import sys
sys.path.insert(0, "./efficientdet")
sys.path.insert(0, "./omegaconf")

import torch
import os
from datetime import datetime
import time
import random
import cv2
import pandas as pd
import numpy as np
import albumentations as A
import matplotlib.pyplot as plt
from albumentations.pytorch.transforms import ToTensorV2
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from glob import glob
from PIL import Image
import xml.etree.ElementTree as elemTree
from torchvision.transforms import functional as F


SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

# Albumentation 

In [16]:
def get_train_transforms():
    return A.Compose(
        [
            A.RandomSizedCrop(min_max_height=(510, 510), height=540, width=960, p=0.5),
            A.OneOf([
                A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit= 0.2, 
                                     val_shift_limit=0.2, p=0.9),
                A.RandomBrightnessContrast(brightness_limit=0.2, 
                                           contrast_limit=0.2, p=0.9),
            ],p=0.9),
            A.ToGray(p=0.01),
            A.HorizontalFlip(p=0.5),
#             A.VerticalFlip(p=0.5),
            A.Resize(height=512, width=512, p=1),
            A.Cutout(num_holes=8, max_h_size=64, max_w_size=64, fill_value=0, p=0.5),
            ToTensorV2(p=1.0),
        ], 
        p=1.0, 
        bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0, 
            min_visibility=0,
            label_fields=['labels']
        )
    )

def get_valid_transforms():
    return A.Compose(
        [
            A.Resize(height=512, width=512, p=1.0),
            ToTensorV2(p=1.0),
        ], 
        p=1.0, 
        bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0, 
            min_visibility=0,
            label_fields=['labels']
        )
    )

# Dataloader 

In [17]:
class Compose(object):
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, image, target):
        for t in self.transforms:
            image, target = t(image, target)
        return image, target


class RandomHorizontalFlip(object):
    def __init__(self, prob):
        self.prob = prob

    def __call__(self, image, target):
        if random.random() < self.prob:
            height, width = image.shape[-2:]
            image = image.flip(-1)
            bbox = target["boxes"]
            bbox[:, [0, 2]] = width - bbox[:, [2, 0]]
            target["boxes"] = bbox
        return image, target


class ToTensor(object):
    def __call__(self, image, target):
        image = F.to_tensor(image)
        return image, target


class CustomDataset(object):
    def __init__(self, root, transforms, phase='train', image_id=None, fine=False):
        self.root = os.path.join(root, phase)
        self.transforms = transforms
        if fine:
            self.labels = sorted(elemTree.parse('./fine_train.xml').findall('image'), key=lambda x: x.attrib['name'])
            self.imgs = sorted(list(glob('./fine_train_img/*jpg')) + list(glob('./fine_train_img/*png')))
        else:
            self.labels = sorted(elemTree.parse(os.path.join(self.root, phase + '.xml')).findall('image'), key=lambda x: x.attrib['name'])
            self.imgs = sorted(list(glob(self.root + '/*jpg') + list(glob(self.root + '/*png'))))
        
        
    def __getitem__(self, idx):
        # 이미지와 마스크를 읽어옵니다
        img_path = self.imgs[idx]
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)
        img /= 255.0
        
#         img = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        boxes = []
        class_names = []
        class_num = {'bus' : 1, 'car' : 2, 'carrier' : 3, 'cat' : 4, 'dog' : 5, 'motorcycle' : 6, 'movable_signage' : 7,
                     'person' : 8, 'scooter' : 9, 'stroller' : 10, 'truck' : 11, 'wheelchair' : 12, 'barricade' : 13, 'bench' : 14, 'chair' : 15,
                     'fire_hydrant' : 16, 'kiosk': 17, 'parking_meter' : 18, 'pole': 19, 'potted_plant' : 20, 'power_controller' : 21, 'stop' : 22, 'table' : 23,
                     'traffic_light_controller':24, 'traffic_sign':25, 'tree_trunk':26, 'bollard':27, 'bicycle' : 28}
                     
        for box_info in label.findall('./box') :
            class_name,x1,y1,x2,y2 = box_info.attrib['label'],box_info.attrib['xtl'],box_info.attrib['ytl'],box_info.attrib['xbr'],box_info.attrib['ybr'] 
            x1,y1,x2,y2 = map(int,map(float, [x1,y1,x2,y2]))
            boxes.append([x1,y1,x2,y2])
            class_names.append(class_num[class_name])
        
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        class_names = torch.as_tensor(class_names, dtype=torch.int64)

        image_id = torch.tensor([idx])

        target = {}
        target["boxes"] = boxes
        target["labels"] = class_names
        target["image_id"] = image_id


        if self.transforms:
            for i in range(10):
                sample = self.transforms(**{
                    'image': img,
                    'bboxes': target['boxes'],
                    'labels': class_names
                })
                if len(sample['bboxes']) > 0:
                    img = sample['image']
                    target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)
                    target['boxes'][:,[0,1,2,3]] = target['boxes'][:,[1,0,3,2]]  #yxyx: be warning
                    break
        
        return img, target, image_id

    def __len__(self):
        return len(self.imgs)

    def get_img_path(self, index) :
        return self.imgs[index]

# def get_transform(train):
#     transforms = []
#     transforms.append(ToTensor())
#     if train:
#         transforms.append(RandomHorizontalFlip(0.5))
#     return Compose(transforms)
                   
def collate_fn(batch):
    return tuple(zip(*batch))
                   
def data_loader(root, batch_size, phase='train', data=None, fine=False) :
    is_train = False
    shuffle = False
    if phase == 'train' :
        is_train = True
        shuffle = True
    if phase == 'train':
        dataset = CustomDataset(root, get_train_transforms(), phase=phase, fine=fine)
    else:
        dataset = CustomDataset(root, get_valid_transforms(), phase=phase)
    dataloader = torch.utils.data.DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle, collate_fn=collate_fn)

    return dataloader

In [18]:
DATASET_PATH = f'../../../datasets/objstrgzip/07_object_detection/07_object_detection'
cuda = True
device = torch.device('cuda') if cuda else torch.device('cpu')

In [19]:
# data_loader_train = data_loader(DATASET_PATH, 2, phase='train')

# Model 

In [20]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [21]:
from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain, DetBenchEval
from effdet.efficientdet import HeadNet

In [22]:
def get_net(retrain):
    config = get_efficientdet_config('tf_efficientdet_d5')
    net = EfficientDet(config, pretrained_backbone=False)
    config.num_classes = 29
    config.image_size = 512
    net.class_net = HeadNet(config, num_outputs=config.num_classes, norm_kwargs=dict(eps=.001, momentum=.01))
    if retrain:
        checkpoint = torch.load('./effdet5-cutmix-augmix/last-checkpoint.bin')
        net.load_state_dict(checkpoint['model_state_dict'])
    return DetBenchTrain(net, config)

net = get_net(retrain=True)

In [23]:
# sample = next(iter(data_loader_train))

In [24]:
# net(sample[0][0][None, :, :, :], sample[1][0]['boxes'][None, :, :], sample[1][0]['labels'][None, :].float())

# Fitter 

In [25]:
import warnings

warnings.filterwarnings("ignore")

class Fitter:
    
    def __init__(self, model, device, config):
        self.config = config
        self.epoch = 0

        self.base_dir = f'./{config.folder}'
        if not os.path.exists(self.base_dir):
            os.makedirs(self.base_dir)
        
        self.log_path = f'{self.base_dir}/log.txt'
        self.best_summary_loss = 10**5

        self.model = model
        self.device = device

        param_optimizer = list(self.model.named_parameters())
        no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
            {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ] 

        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=config.lr)
        self.scheduler = config.SchedulerClass(self.optimizer, **config.scheduler_params)
        self.log(f'Fitter prepared. Device is {self.device}')

    def fit(self, train_loader, validation_loader):
        for e in range(self.config.n_epochs):
            if self.config.verbose:
                lr = self.optimizer.param_groups[0]['lr']
                timestamp = datetime.utcnow().isoformat()
                self.log(f'\n{timestamp}\nLR: {lr}')

            t = time.time()
            summary_loss = self.train_one_epoch(train_loader)

            self.log(f'[RESULT]: Train. Epoch: {self.epoch}, summary_loss: {summary_loss.avg:.5f}, time: {(time.time() - t):.5f}')
            self.save(f'{self.base_dir}/last-checkpoint.bin')

            t = time.time()
            summary_loss = self.validation(validation_loader)

            self.log(f'[RESULT]: Val. Epoch: {self.epoch}, summary_loss: {summary_loss.avg:.5f}, time: {(time.time() - t):.5f}')
            if summary_loss.avg < self.best_summary_loss:
                self.best_summary_loss = summary_loss.avg
                self.model.eval()
                self.save(f'{self.base_dir}/best-checkpoint-{str(self.epoch).zfill(3)}epoch.bin')
                for path in sorted(glob(f'{self.base_dir}/best-checkpoint-*epoch.bin'))[:-3]:
                    os.remove(path)

            if self.config.validation_scheduler:
                self.scheduler.step(metrics=summary_loss.avg)

            self.epoch += 1

    def validation(self, val_loader):
        self.model.eval()
        summary_loss = AverageMeter()
        t = time.time()
        for step, (images, targets, image_ids) in enumerate(val_loader):
            if step==5000: continue
                
            if self.config.verbose:
                if step % self.config.verbose_step == 0:
                    print(
                        f'Val Step {step}/{len(val_loader)}, ' + \
                        f'summary_loss: {summary_loss.avg:.5f}, ' + \
                        f'time: {(time.time() - t):.5f}', end='\r'
                    )
            with torch.no_grad():
                images = torch.stack(images)
                batch_size = images.shape[0]
                images = images.to(self.device).float()
                boxes = [target['boxes'].to(self.device).float() for target in targets]
                labels = [target['labels'].to(self.device).float() for target in targets]

                loss, _, _ = self.model(images, boxes, labels)
                summary_loss.update(loss.detach().item(), batch_size)
                
            ###
            # if step == 50: break
            ###

        return summary_loss

    def train_one_epoch(self, train_loader):
        
        self.model.train()
        summary_loss = AverageMeter()
        t = time.time()
        for step, (images, targets, image_ids) in enumerate(train_loader):
            if self.config.verbose:
                if step % self.config.verbose_step == 0:
                    print(
                        f'Train Step {step}/{len(train_loader)}, ' + \
                        f'summary_loss: {summary_loss.avg:.5f}, ' + \
                        f'time: {(time.time() - t):.5f}', end='\r'
                    )
            
            images = torch.stack(images)
            images = images.to(self.device).float()
            batch_size = images.shape[0]
            boxes = [target['boxes'].to(self.device).float() for target in targets]
            labels = [target['labels'].to(self.device).float() for target in targets]           
            
            self.optimizer.zero_grad()
            
            loss, _, _ = self.model(images, boxes, labels)
            
            loss.backward()

            summary_loss.update(loss.detach().item(), batch_size)

            self.optimizer.step()

            if self.config.step_scheduler:
                self.scheduler.step()
            
            if step % 1000 == 0:
                self.model.eval()
                self.save(f'{self.base_dir}/iter-checkpoint-{loss}loss-{step}step.bin')
                self.model.train()
            
            ###
            #if step == 50: break
            ###    
        return summary_loss
    
    def save(self, path):
        self.model.eval()
        torch.save({
            'model_state_dict': self.model.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'scheduler_state_dict': self.scheduler.state_dict(),
            'best_summary_loss': self.best_summary_loss,
            'epoch': self.epoch,
        }, path)

    def load(self, path):
        checkpoint = torch.load(path)
        self.model.model.load_state_dict(checkpoint['model_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        self.scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        self.best_summary_loss = checkpoint['best_summary_loss']
        self.epoch = checkpoint['epoch'] + 1
        
    def log(self, message):
        if self.config.verbose:
            print(message)
        with open(self.log_path, 'a+') as logger:
            logger.write(f'{message}\n')

In [26]:
class TrainGlobalConfig:
    num_workers = 0
    batch_size = 4
    n_epochs = 10 # n_epochs = 40
    lr = 0.000002

    folder = 'effdet5-cutmix-augmix-4'

    # -------------------
    verbose = True
    verbose_step = 1
    # -------------------

    # --------------------
    step_scheduler = False  # do scheduler.step after optimizer.step
    validation_scheduler = True  # do scheduler.step after validation stage loss

#     SchedulerClass = torch.optim.lr_scheduler.OneCycleLR
#     scheduler_params = dict(
#         max_lr=0.001,
#         epochs=n_epochs,
#         steps_per_epoch=int(len(train_dataset) / batch_size),
#         pct_start=0.1,
#         anneal_strategy='cos', 
#         final_div_factor=10**5
#     )
    
    SchedulerClass = torch.optim.lr_scheduler.ReduceLROnPlateau
    scheduler_params = dict(
        mode='min',
        factor=0.5,
        patience=1,
        verbose=False, 
        threshold=0.0001,
        threshold_mode='abs',
        cooldown=0, 
        min_lr=1e-8,
        eps=1e-08
    )
    # --------------------

In [27]:
def run_training():
    DATASET_PATH = f'../../../datasets/objstrgzip/07_object_detection/07_object_detection'
    device = torch.device('cuda:0')
    net.to(device)

#     train_loader = torch.utils.data.DataLoader(
#         train_dataset,
#         batch_size=TrainGlobalConfig.batch_size,
#         sampler=RandomSampler(train_dataset),
#         pin_memory=False,
#         drop_last=True,
#         num_workers=TrainGlobalConfig.num_workers,
#         collate_fn=collate_fn,
#     )
#     val_loader = torch.utils.data.DataLoader(
#         validation_dataset, 
#         batch_size=TrainGlobalConfig.batch_size,
#         num_workers=TrainGlobalConfig.num_workers,
#         shuffle=False,
#         sampler=SequentialSampler(validation_dataset),
#         pin_memory=False,
#         collate_fn=collate_fn,
#     )
#     data_loader_train = data_loader(DATASET_PATH, TrainGlobalConfig.batch_size, phase='train')
    data_loader_train = data_loader(DATASET_PATH, TrainGlobalConfig.batch_size, phase='train', fine=True)
    data_loader_val = data_loader(DATASET_PATH, 1, phase='val')

    fitter = Fitter(model=net, device=device, config=TrainGlobalConfig)
    fitter.fit(data_loader_train, data_loader_val)

In [28]:
run_training()

Fitter prepared. Device is cuda:0

2020-06-26T13:12:30.989470
LR: 2e-06
[RESULT]: Train. Epoch: 0, summary_loss: 1.39329, time: 34780.83637


error: OpenCV(4.2.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


# Inference 

In [15]:
import gc

In [16]:
def load_ckpt_net(checkpoint_path):
    config = get_efficientdet_config('tf_efficientdet_d5')
    net = EfficientDet(config, pretrained_backbone=False)

    config.num_classes = 29
    config.image_size=512
    net.class_net = HeadNet(config, num_outputs=config.num_classes, norm_kwargs=dict(eps=.001, momentum=.01))

    checkpoint = torch.load(checkpoint_path)
    net.load_state_dict(checkpoint['model_state_dict'])

    del checkpoint
    gc.collect()

    net = DetBenchEval(net, config)
    net.eval();
    return net.cuda()

net = load_ckpt_net('./effdet0-cutmix-augmix/iter-checkpoint-10step.bin')

In [19]:
data_loader_train = data_loader(DATASET_PATH, 2, phase='train')
sample = next(iter(data_loader_train))

In [32]:
images = torch.stack(sample[0]).cuda().float()
predictions = []
with torch.no_grad():
    det = net(images, torch.tensor([1]*images.shape[0]).float().cuda())

In [33]:
det

tensor([[[453.5674, -47.2292,  69.4827, 143.1138,   0.5339,   6.0000],
         [482.7174, -23.8509,  34.7857,  71.5718,   0.5339,   6.0000],
         [453.5228, 416.7641,  69.7747, 143.2081,   0.5339,   6.0000],
         ...,
         [149.6644, -15.3476,  69.6665, 143.3377,   0.5338,   6.0000],
         [325.6643, -15.3495,  69.6666, 143.3416,   0.5338,   6.0000],
         [373.6653, -15.3517,  69.6671, 143.3474,   0.5338,   6.0000]],

        [[453.5673, -47.2299,  69.4829, 143.1152,   0.5339,   6.0000],
         [482.7192, -23.8533,  34.7864,  71.5705,   0.5339,   6.0000],
         [453.5227, 416.7638,  69.7751, 143.2094,   0.5339,   6.0000],
         ...,
         [ 37.6566, -15.3282,  69.6657, 143.3368,   0.5338,   6.0000],
         [389.6651, -15.3475,  69.6653, 143.3481,   0.5338,   6.0000],
         [101.6658, -15.3472,  69.6667, 143.3409,   0.5338,   6.0000]]],
       device='cuda:0')